In [4]:
from pymongo import MongoClient
# from sshtunnel import SSHTunnelForwarder
import pymongo
from tqdm import tqdm
import os
import re

import numpy as np
import pandas as pd

import dateutil.parser
from scipy.stats import entropy

In [5]:
# 使用GPU
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [6]:
! nvidia-smi

Tue Aug  2 23:31:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.129.06   Driver Version: 470.129.06   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 71%   81C    P2   164W / 260W |   3868MiB / 11019MiB |     98%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 31%   

# MongoDB

In [7]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

try:
    conn = pymongo.MongoClient(mongoURI)
    db = conn.eva
    db_de = db.lda_topic_de
    
    
except errors.ConnectionFailure as err:
    print(err)

In [8]:
db_de

Collection(Database(MongoClient(host=['140.117.69.70:30241'], document_class=dict, tz_aware=False, connect=True, authmechanism='SCRAM-SHA-1'), 'eva'), 'lda_topic_de')

# DE

In [18]:
de_topic = db_de.aggregate([
        {"$unwind" : "$classification"},
        {"$sort": {"count":-1}}
])


In [19]:
de_df = pd.DataFrame(de_topic)

In [21]:
# 取cpc前3碼
de_df["cpc_n"] = de_df.classification.apply(lambda x: x[0:3])

In [22]:
de_df

,_id,patentNumber,appDate,company,classification,brief,topic,cpc_n
0,62c6c499afc6a3b84b1e6ddf,DE112012005419B4,2012-10-31 04:00:00,"[GlobalFoundries Inc, GlobalFoundries US Inc]",G06F1/3234,None,6,G06
1,62c6c499afc6a3b84b1e6ddf,DE112012005419B4,2012-10-31 04:00:00,"[GlobalFoundries Inc, GlobalFoundries US Inc]",G06F1/329,None,6,G06
2,62c6c499afc6a3b84b1e6ddf,DE112012005419B4,2012-10-31 04:00:00,"[GlobalFoundries Inc, GlobalFoundries US Inc]",H04W52/0216,None,6,H04
3,62c6c499afc6a3b84b1e6ddf,DE112012005419B4,2012-10-31 04:00:00,"[GlobalFoundries Inc, GlobalFoundries US Inc]",H04L43/08,None,6,H04
4,62c6c499afc6a3b84b1e6ddf,DE112012005419B4,2012-10-31 04:00:00,"[GlobalFoundries Inc, GlobalFoundries US Inc]",H04W52/0219,None,6,H04
...,...,...,...,...,...,...,...,...
8079,62c6c499afc6a3b84b1e7207,DE112020000813T5,2020-02-10 05:00:00,[TDK Corp],H01L41/09,Es wird eine piezoelektrische Zusammensetzung ...,3,H01
8080,62c6c499afc6a3b84b1e7207,DE112020000813T5,2020-02-10 05:00:00,[TDK Corp],H01L41/113,Es wird eine piezoelektrische Zusammensetzung ...,3,H01
8081,62c6c499afc6a3b84b1e7207,DE112020000813T5,2020-02-10 05:00:00,[TDK Corp],H01L41/187,Es wird eine piezoelektrische Zusammensetzung ...,3,H01
8082,62c6c499afc6a3b84b1e7207,DE112020000813T5,2020-02-10 05:00:00,[TDK Corp],H03H9/17,Es wird eine piezoelektrische Zusammensetzung ...,3,H03


In [32]:
de_df2 = de_df.groupby(["cpc_n","topic"],as_index=False).size()

In [33]:
de_df2

,cpc_n,topic,size
0,A01,1,9
1,A01,3,2
2,A01,4,3
3,A01,6,4
4,A01,10,2
...,...,...,...
346,Y04,11,1
347,Y04,12,2
348,Y10,1,1
349,Y10,3,1


## 找出前10的CPC計算各類別的topic比例

In [63]:
de_cpc_ls = ["H04","G06","H01","B60","H05","G01","H03","Y02","G05","H02"]

In [64]:
de_cpc_top10 = de_df2[de_df2.cpc_n.isin(de_cpc_ls)]
de_cpc_top10

,cpc_n,topic,size
70,B60,1,35
71,B60,2,76
72,B60,3,23
73,B60,5,15
74,B60,6,4
...,...,...,...
333,Y02,8,21
334,Y02,9,14
335,Y02,10,15
336,Y02,11,21


In [65]:
de_cpc_sum = de_cpc_top10.groupby(["cpc_n"],as_index=False).sum()[["cpc_n","size"]]
de_cpc_sum.rename(columns = {'size':'sum'}, inplace = True)
de_cpc_sum

,cpc_n,sum
0,B60,358
1,G01,227
2,G05,204
3,G06,1855
4,H01,650
5,H02,163
6,H03,212
7,H04,2714
8,H05,172
9,Y02,212


In [66]:
sum(de_cpc_sum["sum"])

6767

In [67]:
de_cpc_top10_fin = pd.merge(de_cpc_top10, de_cpc_sum, on="cpc_n")
de_cpc_top10_fin

,cpc_n,topic,size,sum
0,B60,1,35,358
1,B60,2,76,358
2,B60,3,23,358
3,B60,5,15,358
4,B60,6,4,358
...,...,...,...,...
116,Y02,8,21,212
117,Y02,9,14,212
118,Y02,10,15,212
119,Y02,11,21,212


In [68]:
# 計算topic比例
de_cpc_top10_fin["topic_ratio"] = de_cpc_top10_fin.apply(lambda x: x["size"]/x["sum"],axis=1)

In [69]:
de_cpc_top10_fin

,cpc_n,topic,size,sum,topic_ratio
0,B60,1,35,358,0.097765
1,B60,2,76,358,0.212291
2,B60,3,23,358,0.064246
3,B60,5,15,358,0.041899
4,B60,6,4,358,0.011173
...,...,...,...,...,...
116,Y02,8,21,212,0.099057
117,Y02,9,14,212,0.066038
118,Y02,10,15,212,0.070755
119,Y02,11,21,212,0.099057


## 計算Entropy
計算各CPC底下topic的混亂程度

In [73]:
entropy(list(de_cpc_top10_fin[de_cpc_top10_fin.cpc_n=="H04"]["topic_ratio"]), base=2)

3.39449366047592

In [74]:
de_entropy_ls = []
for i in de_cpc_ls:
    en_num = entropy(list(de_cpc_top10_fin[de_cpc_top10_fin.cpc_n==i]["topic_ratio"]), base=2)
    de_entropy_ls.append(en_num)

In [75]:
de_entropy_ls

[3.39449366047592,
 3.3879574923490545,
 3.2904399328397425,
 2.492647050401657,
 2.69067471245055,
 3.458391602794772,
 3.189918883610454,
 3.3616411641618713,
 3.0591564299956926,
 2.9508417985352944]

In [77]:
de_entropy = pd.DataFrame((zip(de_cpc_ls, de_entropy_ls)), columns = ['cpc', 'entropy'])
de_entropy

,cpc,entropy
0,H04,3.394494
1,G06,3.387957
2,H01,3.290440
3,B60,2.492647
4,H05,2.690675
5,G01,3.458392
6,H03,3.189919
7,Y02,3.361641
8,G05,3.059156
9,H02,2.950842
